# Import necessary library

In [1]:
import pandas as pd
import spacy

# import data from csv

In [38]:
import os
import platform
import pandas as pd

def check_os_and_load_data(filename):
    if platform.system() == "Windows":
        print("This is a Windows system. Running Windows-specific code.")
        # Assuming the directory path for Windows is '../data/train/'
        path = os.path.join('..', 'data', 'train', filename)
        data = pd.read_csv(path)
        
    elif platform.system() == "Linux":
        print("This is a Linux system. Running Linux-specific code.")
        # Assuming the directory path for Linux is the home directory
        path = os.path.join(os.path.expanduser('~'), 'ownCloud - Michael Saxer (zhaw.ch)@drive.switch.ch', '4 Semester', 'NLP', 'workbench', filename)
        data = pd.read_csv(path)
        
    else:
        print("This is neither a Windows nor a Linux system. You're on your own, sorry.")
        data = None  # Or handle other operating systems as needed
    
    if data is not None:
        if 'id' in data.columns:
            del data['id']
        # data.head()  # Uncomment this if you want to see the first few rows of the dataframe
    return data

# Example usage
# Pass only the filename, not the path.
data = check_os_and_load_data('train.csv')


This is a Linux system. Running Linux-specific code.


In [35]:
data.head(8)



,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0


In [23]:
def fun(x):
    if x.sum() != 0:
        return 1
    return 0

rows = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

data['hate'] = data[rows].apply(lambda x: fun(x), axis=1)

for i in rows:
    del data[i]


In [24]:
data.head(10)

,comment_text,hate
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
5,"""\n\nCongratulations from me as well, use the ...",0
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,Your vandalism to the Matt Shirvington article...,0
8,Sorry if the word 'nonsense' was offensive to ...,0
9,alignment on this subject and which are contra...,0


#  Now comes the fun stuff (trying to build a bert based model pipeline)

import neccesary libs

In [26]:
from transformers import BertModel, BertTokenizer
from tqdm.auto import tqdm
import torch


initalize BERT Tokenizer and Model

In [27]:
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)


now we define the function to tokenize the text

In [28]:
def tokenize_data(text):
    try:
        return tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=512,
            truncation=True,
            padding='max_length',
            return_tensors='pt'
        )
    except Exception as e:
        print(f"Error tokenizing text: {e}")
        return None


apply tokenization for each dataframe one row at the time for memory efficency

In [29]:
# This could take a while depending on the size of your dataset
tqdm.pandas(desc="Tokenizing data")
data['tokenized'] = data['comment_text'].progress_apply(tokenize_data)


Tokenizing data:   0%|          | 0/159571 [00:00<?, ?it/s]

In [32]:
data.head(5)

,comment_text,hate,tokenized
0,Explanation\nWhy the edits made under my usern...,0,"[input_ids, token_type_ids, attention_mask]"
1,D'aww! He matches this background colour I'm s...,0,"[input_ids, token_type_ids, attention_mask]"
2,"Hey man, I'm really not trying to edit war. It...",0,"[input_ids, token_type_ids, attention_mask]"
3,"""\nMore\nI can't make any real suggestions on ...",0,"[input_ids, token_type_ids, attention_mask]"
4,"You, sir, are my hero. Any chance you remember...",0,"[input_ids, token_type_ids, attention_mask]"


now we extract input and attention masks and store them into tensores

In [33]:
# First, we need to drop any rows that failed to tokenize
data = data.dropna(subset=['tokenized'])

# Extract 'input_ids' and 'attention_mask' and create tensors
input_ids = torch.cat(data['tokenized'].apply(lambda x: x['input_ids']).tolist())
attention_mask = torch.cat(data['tokenized'].apply(lambda x: x['attention_mask']).tolist())


now we define the batch sizes

here the pros and cons 

Smaller Batch Sizes:


Memory Efficiency: They require less memory, which means you can train the model even with limited GPU memory.

Regularization Effect: They can have a regularizing effect, helping to prevent overfitting. This is because smaller batches tend to add some noise to the learning process, which means that the model is less likely to learn the noise in the training data.

Frequent Updates: They result in more updates to the model parameters per epoch, which can lead to faster learning initially.

Convergence Quality: They may lead to better generalization in some cases since they explore the error landscape more thoroughly.

Stochastic Nature: They make the optimization process more stochastic, which can help the model to escape local minima.

Larger Batch Sizes:

Computational Efficiency: They make better use of GPU parallelization capabilities, leading to faster computation and training time per epoch.

Stable Gradient Estimates: They provide more accurate estimates of the gradient. However, this can sometimes lead to a model that converges to sharp minima that do not generalize as well.

Less Noise: They add less noise to the learning process, which can be a double-edged sword: it can lead to more stable and reliable convergence but can also result in getting stuck in local minima.

Memory Demands: They require more memory, which can be a limiting factor for machines with less RAM or GPU memory.

Generalization Capability: They might lead to poorer generalization on test data if the model overfits to the training data.



In [40]:
batch_size = 16  # You might need to adjust this depending on your GPU memory


Now twe put the tokenized data trough BERT and create the word embedding

In [41]:
last_hidden_states_batches = []

for i in tqdm(range(0, input_ids.size(0), batch_size), desc='Processing'):
    batch_input_ids = input_ids[i:i+batch_size]
    batch_attention_mask = attention_mask[i:i+batch_size]
    
    with torch.no_grad():
        batch_outputs = model(batch_input_ids, attention_mask=batch_attention_mask)
    
    batch_last_hidden_states = batch_outputs.last_hidden_state
    last_hidden_states_batches.append(batch_last_hidden_states)
    
# Combine the results from all batches
last_hidden_states = torch.cat(last_hidden_states_batches)


Processing:   0%|          | 0/9974 [00:00<?, ?it/s]